In [1]:
import pandas as pd
import requests
import numpy as np
import yfinance as yf
%matplotlib inline
import random
import plotly.io as pio

import datetime as dt

import os
from dotenv import load_dotenv

import datetime as dt
from datetime import timedelta

from stable_baselines3 import PPO

import streamlit as st


pio.templates["custom"] = pio.templates["plotly"]
pio.templates["custom"].layout.font.family = "Cardo"

font_family = "Cardo"

# Set the default template
pio.templates.default = "custom"

In [2]:
from eth_account import Account
from eth_account.signers.local import LocalAccount
from web3 import Web3, EthereumTesterProvider
from web3.exceptions import TransactionNotFound,TimeExhausted

In [3]:
os.chdir('..')

In [4]:
from python_scripts.utils import flipside_api_results, set_random_seed, to_time, clean_prices, calculate_cumulative_return, calculate_cagr, calculate_beta, fetch_and_process_tbill_data,data_cleaning,prepare_data_for_simulation,pull_data,mvo
from sql_scripts.queries import lst_portfolio_prices,eth_btc_prices,dao_advisor_portfolio, yield_portfolio_prices,token_prices
from models.testnet_model import Portfolio
from python_scripts.apis import token_classifier_portfolio

e:\Projects\steth_optimizer\steth_optimizer\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
2024-12-16 23:06:26.497 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-12-16 23:06:26.585 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


Current Directory: e:\Projects\portfolio_optimizers\dao_advisor_project


In [5]:
from diskcache import Cache

In [6]:
from chart_builder.scripts.visualization_pipeline import visualization_pipeline
from chart_builder.scripts.utils import main as chartBuilder

sys.path: ['C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311', 'e:\\Projects\\steth_optimizer\\steth_optimizer', '', 'e:\\Projects\\steth_optimizer\\steth_optimizer\\Lib\\site-packages', 'e:\\projects\\ournetwork\\pipeline', 'e:\\projects\\pipeline_pkg', 'e:\\projects\\chart_builder_pkg', 'e:\\Projects\\steth_optimizer\\steth_optimizer\\Lib\\site-packages\\win32', 'e:\\Projects\\steth_optimizer\\steth_optimizer\\Lib\\site-packages\\win32\\lib', 'e:\\Projects\\steth_optimizer\\steth_optimizer\\Lib\\site-packages\\Pythonwin', 'e:\\projects\\chart_builder_pkg\\chart_builder\\scripts']
Current working directory: e:\Projects\portfolio_optimizers\dao_advisor_project
Current directory: e:\projects\chart_builder_pkg\chart_builder\scripts
Cu

In [7]:
from uniswap import Uniswap

In [8]:
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.cron import CronTrigger
import time


In [9]:
load_dotenv()

# Price Data API
flipside_api_key = os.getenv("FLIPSIDE_API_KEY")

#Blockchain RPCs
GNOSIS_GATEWAY = os.getenv('GNOSIS_GATEWAY')
ARBITRUM_GATEWAY = os.getenv('ARBITRUM_GATEWAY')
OPTIMISM_GATEWAY = os.getenv('OPTIMISM_GATEWAY')
ETHEREUM_GATEWAY = os.getenv('ETHEREUM_GATEWAY')

#Address Credentials
ACCOUNT_ADDRESS = os.getenv('MODEL_ADDRESS')
PRIVATE_KEY = os.getenv('MODEL_KEY')

In [10]:
@st.cache_data(ttl=timedelta(days=7))
def fetch_and_process_tbill_data(api_url, data_key, date_column, value_column, date_format='datetime'):
    api_key = os.getenv("FRED_API_KEY")
    api_url_with_key = f"{api_url}&api_key={api_key}"

    response = requests.get(api_url_with_key)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[data_key])
        
        if date_format == 'datetime':
            df[date_column] = pd.to_datetime(df[date_column])
        
        df.set_index(date_column, inplace=True)
        df[value_column] = df[value_column].astype(float)
        return df
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame in case of failure


2024-12-16 23:06:27.232 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [11]:
three_month_tbill_historical_api = "https://api.stlouisfed.org/fred/series/observations?series_id=TB3MS&file_type=json"

try:
    three_month_tbill = fetch_and_process_tbill_data(three_month_tbill_historical_api, "observations", "date", "value")
    three_month_tbill['decimal'] = three_month_tbill['value'] / 100
    current_risk_free = three_month_tbill['decimal'].iloc[-1]
    print(f"3-month T-bill data fetched: {three_month_tbill.tail()}")
except Exception as e:
    print(f"Error in fetching tbill data: {e}")

2024-12-16 23:06:29.167 
  command:

    streamlit run e:\Projects\steth_optimizer\steth_optimizer\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-16 23:06:29.168 No runtime found, using MemoryCacheStorageManager


3-month T-bill data fetched:            realtime_start realtime_end  value  decimal
date                                                  
2024-07-01     2024-12-12   2024-12-12   5.20   0.0520
2024-08-01     2024-12-12   2024-12-12   5.05   0.0505
2024-09-01     2024-12-12   2024-12-12   4.72   0.0472
2024-10-01     2024-12-12   2024-12-12   4.51   0.0451
2024-11-01     2024-12-12   2024-12-12   4.42   0.0442


In [12]:
params_cache = Cache('test_model_cache')

In [13]:
today_utc = dt.datetime.now(dt.timezone.utc) - timedelta(hours=5)
formatted_today_utc = today_utc.strftime('%Y-%m-%d %H:00:00')

In [14]:
end_date = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00') 

In [15]:
print(list(params_cache.iterkeys()))

['Savings Stables Model_2 Test Start', 'arbitrum_classifier Assets', 'arbitrum_classifier Classifier', 'arbitrum_classifier Params', 'arbitrum_classifier Test Start', 'ethereum_classifier Assets', 'ethereum_classifier Test Start', 'test_arbitrum Assets', 'to_save_cache Assets']


In [97]:
model_name = 'arbitrum_classifier'
cache = Cache(f'live_{model_name}_cache')
normalized_value = 100

clear_cache = True
 
params = params_cache.get(f'{model_name} Params')
network = params['network']
chain = params['network']
print(f'model_name: {model_name} \nnetwork: {network}')

test_start_date = params_cache.get(f'{model_name} Test Start') # First hour after end of training dataset; ensures no overlap
filtered_assets = params_cache.get(f'{model_name} Assets')
classifier_data = params_cache.get(f'{model_name} Classifier')

days = params['days']
function = params['function']
seed = params['seed']

rebalance_frequency = params['rebalance_frequency']

test_start_date = str(test_start_date)
start_date = formatted_today_utc
print(f'test_start_date: {test_start_date}')

diff_tolerance_threshold = 0.05 #percent for diff between new portfolio action and current portfolio


model_name: arbitrum_classifier 
network: arbitrum
test_start_date: 2024-10-31 12:00:00+00:00


In [98]:
if clear_cache:
    cache.clear() 

In [18]:
historical_data = cache.get(f'{model_name} historical_data', pd.DataFrame())
historical_port_values = cache.get(f'{model_name} historical_port_values', pd.DataFrame())
oracle_prices = cache.get(f'{model_name} oracle_prices',pd.DataFrame())
last_rebalance_time = cache.get(f'{model_name} last_rebalance_time', None)
model_actions = cache.get(f'{model_name} actions', pd.DataFrame()) 

In [19]:
def prices_data_func(network,
                     api_key,use_cached_data,name,days=None,
                     function=None,start_date=None,
                     backtest_period=None):
    
    if start_date is None and backtest_period is None:
        raise KeyError("Provide either a start date or backtest_period")
    
    print(f"backtest days: {(pd.to_datetime(dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00')) - pd.to_datetime(start_date)).days}")
    
    if backtest_period is None:
        backtest_period = (pd.to_datetime(dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00')) - pd.to_datetime(start_date)).days * 24
        if backtest_period < 1:
            backtest_period = 1

    if function is None:

        data = token_classifier_portfolio(
            network=network,
            days=days,
            name=name,
            api_key = api_key,
            use_cached_data=use_cached_data,
            start_date = start_date,
            prices_only=True
        )

        prices_df = data_cleaning(data['portfolio'])
        prices_df
    else: 
        data = pull_data(function=function,start_date=start_date, path=f'data/{name}.csv', api=not use_cached_data,model_name=name)
        prices_df = data_cleaning(data['portfolio'])
        prices_df = prices_df[prices_df.index >= start_date].dropna()
        prices_df

    # prices_df.columns = prices_df.columns.str.replace('_Price','')
    filtered_assets_with_price = [f"{asset}_Price" for asset in filtered_assets]


    return data, prices_df[filtered_assets_with_price]

In [21]:
data, prices_df = prices_data_func(
                    network=network, 
                    name=model_name,
                    api_key=flipside_api_key,
                    use_cached_data=False,
                    function=function,
                    start_date=start_date,
                    )
prices_df = prepare_data_for_simulation(prices_df, start_date, end_date)


2024-12-16 23:06:30.806 No runtime found, using MemoryCacheStorageManager


backtest days: 0
use_cached_data: False
volume_threshold: 1
start_date: 2024-12-16 23:00:00
prices_start: 2024-12-16 23:00:00
Beginning: '2024-12-16 23:00:00'


Query not completed. Retrying in 30 seconds...
cleaning prices
cleaned prices:                        hour  ARB_Price  CRV_Price  GMX_Price  GRT_Price  \
0  2024-12-16T23:00:00.000Z   1.027000      1.092      37.96   0.280871   
1  2024-12-17T00:00:00.000Z   1.020000      1.082      37.58   0.278554   
2  2024-12-17T01:00:00.000Z   1.006000      1.073      37.30   0.275907   
3  2024-12-17T02:00:00.000Z   0.986978      1.071      36.76   0.273421   

   LINK_Price  MAGIC_Price  PENDLE_Price  UNI_Price  WBTC_Price  WETH_Price  
0       29.08     0.595563          6.37      16.55    105775.0     4043.24  
1       29.08     0.590684          6.38      16.45    105602.0     4028.73  
2       28.84     0.584080          6.30      16.29    105600.0     3989.99  
3       28.65     0.575840          6.19      15.99    105641.0     3965.85  
DatetimeIndex(['2024-12-16 23:00:00+00:00', '2024-12-17 00:00:00+00:00',
               '2024-12-17 01:00:00+00:00', '2024-12-17 02:00:00+00:00'],
        

e:\Projects\portfolio_optimizers\dao_advisor_project\python_scripts\utils.py:307: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

e:\Projects\portfolio_optimizers\dao_advisor_project\python_scripts\utils.py:314: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [27]:
params_cache.get(f'{model_name} Classifier')

,symbol,token_address,latest_hour,latest_price,sixty_day_price,sixty_d_return,rolling_7d_avg,rolling_30d_avg,volume,average_order,__row_index
0,CRV,0x11cdb42b0eb46d95f990bedd4695a6e3fa034978,2024-12-16 02:00:00+00:00,1.15,0.29075,2.955288,1.129338,0.752708,5.206880e+07,625.902174,0
1,LINK,0xf97f4df75117a78c1a5a0dbb814af92458539fb4,2024-12-16 02:00:00+00:00,29.34,13.84000,1.119942,26.477811,20.614286,1.569272e+08,1008.179565,1
2,UNI,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,2024-12-16 02:00:00+00:00,17.31,8.54000,1.026932,16.874615,13.356671,2.370339e+07,538.799173,2
9,WBTC,0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f,2024-12-16 02:00:00+00:00,105146.00,90781.00000,0.158238,99561.781065,96637.266297,2.611187e+09,2823.725555,9


In [28]:
for key in data.keys():
    print(f'{key}: {data[key]}')


classifier:    symbol                               token_address  \
0     CRV  0x11cdb42b0eb46d95f990bedd4695a6e3fa034978   
1    LINK  0xf97f4df75117a78c1a5a0dbb814af92458539fb4   
2     UNI  0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0   
3     ARB  0x912ce59144191c1204e64559fe8253a0e49e6548   
4   MAGIC  0x539bde0d7dbd336b79148aa742883198bbf60342   
5     GRT  0x9623063377ad1b27544c965ccd7342f7ea7e88c7   
6     GMX  0xfc5a1a6eb076a2c7ad06ed22c90d7e710e35ad0a   
7    WETH  0x82af49447d8a07e3bd95bd0d56f35241523fbab1   
8  PENDLE  0x0c880f6761f1af8d9aa9c466984b80dab9a8c9e8   
9    WBTC  0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f   

                latest_hour   latest_price  sixty_day_price  sixty_d_return  \
0 2024-12-16 02:00:00+00:00       1.150000         0.290750        2.955288   
1 2024-12-16 02:00:00+00:00      29.340000        13.840000        1.119942   
2 2024-12-16 02:00:00+00:00      17.310000         8.540000        1.026932   
3 2024-12-16 02:00:00+00:00       1.033000  

#  Should only need to modify the router and factory contract addresses. The best way would probably be to add the ability to override the contract addresses using constructor arguments.

- How to switch to Honeyswap on Gnosis

In [29]:
data.keys()

dict_keys(['classifier', 'portfolio', 'data start', 'latest_hour', 'original_hour'])

In [30]:
data['data start']

'2024-06-16 14:00:00'

In [31]:
data['latest_hour']

Timestamp('2024-12-16 02:00:00+0000', tz='UTC')

In [33]:
portfolio = classifier_data[['symbol','token_address']]
portfolio

,symbol,token_address
0,CRV,0x11cdb42b0eb46d95f990bedd4695a6e3fa034978
1,LINK,0xf97f4df75117a78c1a5a0dbb814af92458539fb4
2,UNI,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0
9,WBTC,0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f


In [34]:
classifier_fig1 = visualization_pipeline(
    df=classifier_data,
    groupby='symbol',
    num_col='sixty_d_return',
    chart_type='ranked bar',
    title='Sixty Day Return by Symbol',
    ticksuffix=dict(y1='%',y2=None),
    to_percent=True,
    text=True

)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: min
turn_to_time: False
df sort order at init: ['CRV' 'LINK' 'UNI' 'WBTC']


In [36]:
chartBuilder(
    classifier_fig1,
    title_xy=dict(x=0.6,y=0.8),
    title=f'{network.upper()} Tokens Classifier <br> {days}D Return',
    dt_index=False,
    add_the_date=False,
    save=False,
    show=False
)

save:False
logo path: ../img/Logos/None
Generating ranked bar plot...
sort_list vis pipe: True
df sort order before ranked bar funct: ['CRV' 'LINK' 'UNI' 'WBTC']
combined_colors: ['#2E91E5', '#E15F99', '#1CA71C', '#FB0D0D', '#DA16FF', '#222A2A', '#B68100', '#750D86', '#EB663B', '#511CFB', '#00A08B', '#FB00D1', '#FC0080', '#B2828D', '#6C7C32', '#778AAE', '#862A16', '#A777F1', '#620042', '#1616A7', '#DA60CA', '#6C4516', '#0D2A63', '#AF0038', 'rgb(141,211,199)', 'rgb(255,255,179)', 'rgb(190,186,218)', 'rgb(251,128,114)', 'rgb(128,177,211)', 'rgb(253,180,98)', 'rgb(179,222,105)', 'rgb(252,205,229)', 'rgb(217,217,217)', 'rgb(188,128,189)', 'rgb(204,235,197)', 'rgb(255,237,111)', '#FECB52', '#FF97FF', '#B6E880', '#FF6692', '#19D3F3', '#FFA15A', '#AB63FA', '#00CC96', '#EF553B', '#636EFA', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf', '#999999'

In [37]:
prices_df

,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
2024-12-16 23:00:00,105775.0,16.55,29.08,1.092
2024-12-17 00:00:00,105602.0,16.45,29.08,1.082
2024-12-17 01:00:00,105600.0,16.29,28.84,1.073
2024-12-17 02:00:00,105641.0,15.99,28.65,1.071
2024-12-17 03:00:00,105641.0,15.99,28.65,1.071
2024-12-17 04:00:00,105641.0,15.99,28.65,1.071


In [38]:
erc20_abi = [
    {
        "constant": False,
        "inputs": [
            {"name": "_to", "type": "address"},
            {"name": "_value", "type": "uint256"}
        ],
        "name": "transfer",
        "outputs": [{"name": "", "type": "bool"}],
        "type": "function"
    }
]

def network(chain):
    if chain == 'gnosis':
        primary_gateway = GNOSIS_GATEWAY  # Replace with your Infura URL
        backup_gateway = 'https://lb.nodies.app/v1/406d8dcc043f4cb3959ed7d6673d311a'  # Your backup gateway
    elif chain == 'arbitrum':
        primary_gateway = ARBITRUM_GATEWAY  # Replace with your Infura URL
        backup_gateway = ARBITRUM_GATEWAY
    elif chain == 'optimism':
        primary_gateway = OPTIMISM_GATEWAY  # Replace with your Infura URL
        backup_gateway = OPTIMISM_GATEWAY
    elif chain == 'ethereum':
        primary_gateway = ETHEREUM_GATEWAY  # Replace with your Infura URL
        backup_gateway = ETHEREUM_GATEWAY

    print(f'Gateway: {primary_gateway}')

    for gateway in [primary_gateway, backup_gateway]:
        w3 = Web3(Web3.HTTPProvider(gateway))
        if w3.is_connected():
            try:
                latest_block = w3.eth.get_block('latest')['number']  # Only try this if connected
                print(f"Connected to {chain} via {gateway}: {latest_block} block")
                return w3, gateway
            except Exception as e:
                print(f"Connected to {gateway} but failed to fetch latest block. Error: {e}")
        else:
            print(f"Failed to connect to {chain} via {gateway}. Trying next gateway...")

    raise ConnectionError(f"Failed to connect to {chain} network using both primary and backup gateways.")



w3, gateway = network(chain)

account = Account.from_key(PRIVATE_KEY)
w3.eth.default_account = account.address

print(f"Connected account: {account.address}")

Gateway: https://arb-mainnet.g.alchemy.com/v2/sDx1aRO1I82_YWxxUK9S_jnDXHTXEqFl
Connected to arbitrum via https://arb-mainnet.g.alchemy.com/v2/sDx1aRO1I82_YWxxUK9S_jnDXHTXEqFl: 285580868 block
Connected account: 0x75baD5ae9f46e8AEBf61e4A7179cEf3A0CeD6766


In [39]:
TOKEN_CONTRACTS = {
    row['symbol']: row['token_address'] for _, row in portfolio.iterrows()
}

TOKEN_CONTRACTS

{'CRV': '0x11cdb42b0eb46d95f990bedd4695a6e3fa034978',
 'LINK': '0xf97f4df75117a78c1a5a0dbb814af92458539fb4',
 'UNI': '0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0',
 'WBTC': '0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f'}

In [40]:
def get_token_decimals(TOKEN_CONTRACTS):
    """Fetch decimals for each token contract using Web3."""
    try:
        # ERC20 ABI for decimals function
        decimals_abi = [
            {
                "constant": True,
                "inputs": [],
                "name": "decimals",
                "outputs": [{"name": "", "type": "uint8"}],
                "type": "function"
            }
        ]

        # Dictionary to store decimals for each token
        token_decimals = {}

        for token, address in TOKEN_CONTRACTS.items():
            if address:
                try:
                    contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=decimals_abi)
                    decimals = contract.functions.decimals().call()
                    token_decimals[token] = decimals
                except Exception as e:
                    print(f"Error fetching decimals for {token}: {e}")
                    token_decimals[token] = None
            else:
                print(f"Contract address for {token} is not set.")
                token_decimals[token] = None

        return token_decimals

    except Exception as e:
        print(f"Error: {e}")
        return None

In [41]:
def get_balance(TOKEN_CONTRACTS, TOKEN_DECIMALS, ACCOUNT_ADDRESS):
    """Fetch token balances using Web3 with provided decimal adjustments."""
    try:
        # ERC20 ABI for balanceOf function
        erc20_abi = [
            {
                "constant": True,
                "inputs": [{"name": "_owner", "type": "address"}],
                "name": "balanceOf",
                "outputs": [{"name": "balance", "type": "uint256"}],
                "type": "function"
            }
        ]

        # Fetch balances using the provided decimals
        balances = {}
        for token, address in TOKEN_CONTRACTS.items():
            decimals = TOKEN_DECIMALS.get(token)

            if address and decimals is not None:
                try:
                    contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=erc20_abi)
                    balance_wei = contract.functions.balanceOf(ACCOUNT_ADDRESS).call()
                    balances[token] = balance_wei / 10**decimals
                except Exception as e:
                    print(f"Error fetching balance for {token}: {e}")
                    balances[token] = None
            else:
                print(f"Skipping {token} due to missing address or decimals.")
                balances[token] = None

        # Print and return balances
        print(f"Balances for account {ACCOUNT_ADDRESS}: {balances}")
        return balances

    except Exception as e:
        print(f"Error fetching balances: {e}")
        return None

In [42]:
def convert_to_usd(balances, prices,TOKEN_CONTRACTS):
    """
    Convert token balances to their USD equivalent using token prices.

    Parameters:
    - balances (dict): Dictionary of token balances.
    - prices (dict): Dictionary of token prices.

    Returns:
    - dict: Dictionary of token balances converted to USD.
    """
    # Convert token keys to upper case for consistency
    print(f'balances: {balances.keys()}')
    print(f'TOKEN_CONTRACTS.keys(): {TOKEN_CONTRACTS.keys()}')

    for token in TOKEN_CONTRACTS.keys():
        if f"{token}" not in prices:
            print(f"Missing price for token: {token}")

    usd_balances = {
        token: balances[token] * prices[f"{token}"]
        for token in TOKEN_CONTRACTS.keys()
        if f"{token}" in prices
    }
    return usd_balances

In [43]:
def calculate_compositions(balances_usd, total_balance):
    """
    Calculate the composition of each token in the portfolio.

    Parameters:
    - balances_usd (dict): Dictionary of token balances in USD.
    - total_balance (float): Total portfolio balance in USD.

    Returns:
    - dict: Dictionary of token compositions.
    """
    if total_balance == 0:
        print("Warning: Total portfolio balance is zero. Setting all compositions to 0.")
        return {f"{token} comp": 0.0 for token in balances_usd}

    # Calculate composition for each token
    compositions = {f"{token} comp": balance_usd / total_balance for token, balance_usd in balances_usd.items()}
    return compositions

In [44]:
def update_historical_data(live_comp):
    global historical_data
    new_data = pd.DataFrame([live_comp])
    historical_data = pd.concat([historical_data, new_data]).reset_index(drop=True)
    historical_data.drop_duplicates(subset='date', keep='last', inplace=True)
    cache.set(f'{model_name} historical_data', historical_data)

def update_portfolio_data(values):
    global historical_port_values
    print(f'values: {values}')
    values = pd.DataFrame([values])
    historical_port_values = pd.concat([historical_port_values, values]).reset_index(drop=True)
    historical_port_values.drop_duplicates(subset='date', keep='last', inplace=True)
    cache.set(f'{model_name} historical_port_values', historical_port_values)

def update_price_data(values):
    global oracle_prices

    # Ensure the 'hour' column exists by resetting index if necessary
    if isinstance(values.index, pd.DatetimeIndex):
        values = values.reset_index().rename(columns={'index': 'hour'})
    
    if 'hour' not in values.columns:
        raise ValueError("The provided DataFrame must have a 'hour' column.")

    # Concatenate the new values with the existing oracle_prices
    oracle_prices = pd.concat([oracle_prices, values]).drop_duplicates(subset='hour', keep='last').reset_index(drop=True)
    
    # Cache the updated oracle_prices
    cache.set(f'{model_name} oracle_prices', oracle_prices)

    print(f'Updated oracle_prices:\n{oracle_prices}')

def update_model_actions(actions):
    global model_actions
    print(f'model actions before update: {model_actions}')
    new_data = pd.DataFrame(actions)
    print(f'new data: {new_data}')
    model_actions = pd.concat([model_actions, new_data]).reset_index(drop=True)
    model_actions.drop_duplicates(subset='Date', keep='last', inplace=True)
    cache.set(f'{model_name} actions', model_actions)

In [45]:
TOKEN_DECIMALS = get_token_decimals(TOKEN_CONTRACTS)

In [46]:
model_balances = get_balance(TOKEN_CONTRACTS,TOKEN_DECIMALS,ACCOUNT_ADDRESS)

Balances for account 0x75baD5ae9f46e8AEBf61e4A7179cEf3A0CeD6766: {'CRV': 4.291008269e-09, 'LINK': 1.652236831399549, 'UNI': 0.18303746793932313, 'WBTC': 0.0}


In [48]:
TOKEN_CONTRACTS.keys()

dict_keys(['CRV', 'LINK', 'UNI', 'WBTC'])

In [49]:
latest_prices = {
    token: float(prices_df[f"{token}_Price"].iloc[-1])
    for token in TOKEN_CONTRACTS.keys()
    if f"{token}_Price" in prices_df.columns
}

latest_prices


{'CRV': 1.071, 'LINK': 28.65, 'UNI': 15.99, 'WBTC': 105641.0}

In [51]:
update_price_data(prices_df)
oracle_prices

Updated oracle_prices:
                 hour  WBTC_Price  UNI_Price  LINK_Price  CRV_Price
0 2024-12-16 07:00:00    104384.0      16.70       29.04      1.099
1 2024-12-16 22:00:00    105760.0      16.60       29.66      1.099
2                 NaT    105641.0      15.99       28.65      1.071
3 2024-12-16 23:00:00    105775.0      16.55       29.08      1.092
4 2024-12-17 00:00:00    105602.0      16.45       29.08      1.082
5 2024-12-17 01:00:00    105600.0      16.29       28.84      1.073
6 2024-12-17 02:00:00    105641.0      15.99       28.65      1.071
7 2024-12-17 03:00:00    105641.0      15.99       28.65      1.071
8 2024-12-17 04:00:00    105641.0      15.99       28.65      1.071


,hour,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
0,2024-12-16 07:00:00,104384.0,16.70,29.04,1.099
1,2024-12-16 22:00:00,105760.0,16.60,29.66,1.099
2,NaT,105641.0,15.99,28.65,1.071
3,2024-12-16 23:00:00,105775.0,16.55,29.08,1.092
4,2024-12-17 00:00:00,105602.0,16.45,29.08,1.082
5,2024-12-17 01:00:00,105600.0,16.29,28.84,1.073
6,2024-12-17 02:00:00,105641.0,15.99,28.65,1.071
7,2024-12-17 03:00:00,105641.0,15.99,28.65,1.071
8,2024-12-17 04:00:00,105641.0,15.99,28.65,1.071


In [52]:
model_balances

{'CRV': 4.291008269e-09,
 'LINK': 1.652236831399549,
 'UNI': 0.18303746793932313,
 'WBTC': 0.0}

In [53]:
latest_prices

{'CRV': 1.071, 'LINK': 28.65, 'UNI': 15.99, 'WBTC': 105641.0}

In [54]:
latest_prices['WBTC'] * model_balances['WBTC']

0.0

In [55]:
model_balances_usd = convert_to_usd(model_balances,latest_prices,TOKEN_CONTRACTS)
portfolio_balance = sum(model_balances_usd.values())

balances: dict_keys(['CRV', 'LINK', 'UNI', 'WBTC'])
TOKEN_CONTRACTS.keys(): dict_keys(['CRV', 'LINK', 'UNI', 'WBTC'])


In [56]:
model_balances_usd

{'CRV': 4.595669856099e-09,
 'LINK': 47.336585219597076,
 'UNI': 2.9267691123497768,
 'WBTC': 0.0}

In [57]:
comp_dict = {
    f"{token} comp": balance_usd / portfolio_balance
    for token, balance_usd in model_balances_usd.items()
}

comp_dict["date"] = formatted_today_utc

In [58]:
model_balances_usd.values()

dict_values([4.595669856099e-09, 47.336585219597076, 2.9267691123497768, 0.0])

In [59]:
comp_dict

{'CRV comp': 9.143181780762791e-11,
 'LINK comp': 0.9417713132046657,
 'UNI comp': 0.05822868670390256,
 'WBTC comp': 0.0,
 'date': '2024-12-16 23:00:00'}

In [60]:
update_historical_data(comp_dict)

historical_data

,CRV comp,LINK comp,UNI comp,WBTC comp,date
0,9.239875e-11,0.940108,0.059892,0.0,2024-12-16 07:00:00
1,9.143182e-11,0.941771,0.058229,0.0,2024-12-16 22:00:00
3,9.143182e-11,0.941771,0.058229,0.0,2024-12-16 23:00:00


In [61]:
portfolio_dict = {
    "Portfolio Value": portfolio_balance,
    "date": formatted_today_utc
}

update_portfolio_data(portfolio_dict)
historical_port_values

values: {'Portfolio Value': 50.26335433654252, 'date': '2024-12-16 23:00:00'}


,Portfolio Value,date
0,51.037683,2024-12-16 07:00:00
1,50.263354,2024-12-16 22:00:00
2,50.263354,2024-12-17 03:00:00
4,50.263354,2024-12-16 23:00:00


# DEX Swap

In [62]:
network = params['network']

In [63]:
#Gnosis - uses version 2
#Arbitrum - uses version 3
#Ethereum is the default; using version 3

if network == 'gnosis':
    factory = '0xA818b4F111Ccac7AA31D0BCc0806d64F2E0737D7'
    router = '0x1C232F01118CB8B424793ae03F870aa7D0ac7f77'
    version = 2
elif network == 'arbitrum':
    factory = '0x1F98431c8aD98523631AE4a59f267346ea31F984'
    router = '0x5E325eDA8064b456f4781070C0738d849c824258'
    version = 3
elif network == 'ethereum':
    factory = None
    router = None
    version = 3

In [64]:
uniswap = Uniswap(address=ACCOUNT_ADDRESS, private_key=PRIVATE_KEY, version=version, provider=gateway,router_contract_addr=router,factory_contract_addr=factory)

2024-12-16 23:07:10,154 INFO uniswap.uniswap Using <web3.main.Web3 object at 0x0000021A15DFE410> ('arbitrum', netid: 42161)
2024-12-16 23:07:10,474 INFO uniswap.uniswap Using factory contract: <web3._utils.datatypes.Contract object at 0x0000021A16572B90>


In [65]:
portfolio

,symbol,token_address
0,CRV,0x11cdb42b0eb46d95f990bedd4695a6e3fa034978
1,LINK,0xf97f4df75117a78c1a5a0dbb814af92458539fb4
2,UNI,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0
9,WBTC,0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f


In [66]:
filtered_assets

['WBTC', 'UNI', 'LINK', 'CRV']

In [67]:
TOKEN_CONTRACTS

{'CRV': '0x11cdb42b0eb46d95f990bedd4695a6e3fa034978',
 'LINK': '0xf97f4df75117a78c1a5a0dbb814af92458539fb4',
 'UNI': '0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0',
 'WBTC': '0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f'}

In [68]:
TOKEN_DECIMALS

{'CRV': 18, 'LINK': 18, 'UNI': 18, 'WBTC': 8}

In [69]:
model_balances

{'CRV': 4.291008269e-09,
 'LINK': 1.652236831399549,
 'UNI': 0.18303746793932313,
 'WBTC': 0.0}

In [70]:
model_balances_usd

{'CRV': 4.595669856099e-09,
 'LINK': 47.336585219597076,
 'UNI': 2.9267691123497768,
 'WBTC': 0.0}

In [71]:
import time
import streamlit as st

coingecko_key = os.getenv('COINGECKO_KEY')

@st.cache_data(ttl=timedelta(hours=1))
def coingecko_api():

    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": coingecko_key
    }

    base_url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids=ethereum"
    
    response = requests.get(base_url, headers=headers)

    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        return None

    data = response.json()
    df = pd.DataFrame(data)
    df.to_csv('data/list_data.csv', index=False)

    return df

2024-12-16 23:07:10.638 No runtime found, using MemoryCacheStorageManager


In [72]:
eth_data = coingecko_api()
latest_eth_price = eth_data['current_price'].values[0]
eth_data[['symbol','current_price','last_updated']]

2024-12-16 23:07:10.671 No runtime found, using MemoryCacheStorageManager


,symbol,current_price,last_updated
0,eth,4036.61,2024-12-17T03:53:51.636Z


In [73]:
# tokens = ['0x75baD5ae9f46e8AEBf61e4A7179cEf3A0CeD6766'] # WETH Price
# token_prices_query = token_prices(tokens,network,start_date)
# tokens_df = flipside_api_results(token_prices_query,flipside_api_key)

In [74]:
WETH_ADDRESS = '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'
checksum_weth_address = Web3.to_checksum_address(WETH_ADDRESS)

In [75]:
import pandas as pd
from uniswap import Uniswap
from web3 import Web3
import math

def rebalance_portfolio(
    uniswap, 
    token_contracts, 
    token_decimals, 
    target_compositions, 
    account_address, 
):
    """
    Rebalances the portfolio by selling all tokens into WETH and then buying target allocations using WETH.

    Parameters:
    - uniswap: Initialized Uniswap class instance.
    - token_contracts: Dict of token addresses.
    - token_decimals: Dict of token decimals.
    - target_compositions: Dict of target compositions as fractions summing to 1.
    - account_address: ETH wallet address.
    - web3: Initialized Web3 instance.
    """

    # WETH address and checksum
    WETH_ADDRESS = '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'
    checksum_weth_address = Web3.to_checksum_address(WETH_ADDRESS)

    # Step 1: Convert Token Addresses to Checksum Format
    checksum_addresses = {token: Web3.to_checksum_address(address) for token, address in token_contracts.items()}

    # Step 2: Sell All Current Token Holdings into WETH
    for token, address in checksum_addresses.items():
        try:
            balance_wei = uniswap.get_token_balance(address)
            balance = balance_wei / 10**token_decimals[token]
            
            # Adjust the balance to avoid precision issues (round down to 6 decimal places)
            adjusted_balance = math.floor(balance * 10**8) / 10**8
            
            if adjusted_balance > 0:
                amount_to_sell = int(adjusted_balance * 10**token_decimals[token])
                print(f"Selling {adjusted_balance:.6f} {token} for WETH")
                uniswap.make_trade(
                    checksum_addresses[token],
                    checksum_weth_address,  # WETH as output token
                    amount_to_sell
                )
                wait_time = random.randint(15, 30)
                print(f"Waiting {wait_time} seconds before the next call...")
                time.sleep(wait_time)
        except Exception as e:
            print(f"Error selling {token}: {e}")

    # Step 3: Get Current WETH Balance
    weth_balance_wei = uniswap.get_token_balance(checksum_weth_address)
    weth_balance = weth_balance_wei / 10**18
    print(f"Total WETH balance after selling: {weth_balance:.6f} WETH")

    # Step 4: Buy Target Tokens Based on Target Compositions
    for token, target_weight in target_compositions.items():
        if target_weight > 0:
            weth_to_spend = weth_balance * target_weight
            
            # Adjust the WETH amount to avoid precision issues (round down to 6 decimal places)
            adjusted_weth_to_spend = math.floor(weth_to_spend * 10**8) / 10**8

            if adjusted_weth_to_spend <= 0:
                continue

            try:
                print(f"Buying {token} with {adjusted_weth_to_spend:.6f} WETH")

                uniswap.make_trade(
                    checksum_weth_address,        # WETH as input token
                    checksum_addresses[token],    # Target token
                    int(adjusted_weth_to_spend * 10**18),  # Convert WETH amount to wei
                    fee=3000                      # Assuming 0.3% fee pool for Uniswap V3
                )

                wait_time = random.randint(15, 30)
                print(f"Waiting {wait_time} seconds before the next call...")
                time.sleep(wait_time)

            except Exception as e:
                print(f"Error buying {token}: {e}")

    # Step 5: Log the Rebalancing Info
    final_weth_balance = uniswap.get_token_balance(checksum_weth_address) / 10**18
    print(f"Final WETH balance: {final_weth_balance:.6f} WETH")

    rebal_info = {
        "account_address": account_address,
        "initial_weth_balance": weth_balance,
        "final_weth_balance": final_weth_balance,
        "purchases": target_compositions,
    }

    # Save rebalancing info to CSV
    rebal_df = pd.DataFrame([rebal_info])
    rebal_df.to_csv('data/live_rebal_results.csv', index=False)
    print("Rebalancing info saved to 'data/live_rebal_results.csv'.")


In [76]:
latest_prices

{'CRV': 1.071, 'LINK': 28.65, 'UNI': 15.99, 'WBTC': 105641.0}

In [77]:
latest_eth_price

4036.61

In [78]:
latest_prices

{'CRV': 1.071, 'LINK': 28.65, 'UNI': 15.99, 'WBTC': 105641.0}

In [79]:
TOKEN_CONTRACTS

{'CRV': '0x11cdb42b0eb46d95f990bedd4695a6e3fa034978',
 'LINK': '0xf97f4df75117a78c1a5a0dbb814af92458539fb4',
 'UNI': '0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0',
 'WBTC': '0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f'}

In [80]:
def should_rebalance(current_time, actions_df, rebalancing_frequency):
    global last_rebalance_time

    # Convert current_time to a Timestamp object
    current_time = pd.to_datetime(current_time).replace(tzinfo=None, minute=0, second=0, microsecond=0)

    print(f'last rebal time: {last_rebalance_time}')
    print(f'current time {current_time}')
    print(f'actions df {actions_df}')
    print(f'rebalancing frequency {rebalancing_frequency}')

    print(f'should rebal current time: {current_time}')

    if rebalancing_frequency == 1:
        # Check if last_rebalance_time is None (first rebalance) or if more than an hour has passed
        if last_rebalance_time is None or (current_time - last_rebalance_time).total_seconds() >= 3600:
            last_rebalance_time = current_time
            cache.set('last_rebalance_time', last_rebalance_time)
            print(f'last_rebalance_time updated to: {last_rebalance_time}')
            return True
        else:
            print("Rebalancing is not required at this time.")
            return False
    else:
        actions_df = actions_df.sort_values(by='Date')
        print(f'actions df: {actions_df}')

        last_rebalance_time_from_actions = pd.to_datetime(actions_df['Date'].iloc[-1])

        # Ensure last_rebalance_time_from_actions is tz-naive and truncate to date and hour
        last_rebalance_time_from_actions = last_rebalance_time_from_actions.replace(tzinfo=None, minute=0, second=0, microsecond=0)
        print(f'should rebal last rebal time: {last_rebalance_time_from_actions}')

        # Calculate hours since last rebalance
        hours_since_last_rebalance = (current_time - last_rebalance_time_from_actions).total_seconds() / 3600
        print(f'hours since last rebal: {hours_since_last_rebalance}')
        
        if hours_since_last_rebalance == 0:
            print('initial rebalance')
            if last_rebalance_time is None or (current_time - last_rebalance_time).total_seconds() >= 3600:
                last_rebalance_time = current_time
                cache.set('last_rebalance_time', last_rebalance_time)
                print(f'last_rebalance_time updated to: {last_rebalance_time}')
                return True
            else:
                print("Rebalancing is not required at this time.")
                return False
        elif hours_since_last_rebalance >= rebalancing_frequency:
            print("Rebalancing required based on frequency.")
            if last_rebalance_time is None or (current_time - last_rebalance_time).total_seconds() >= rebalancing_frequency * 3600:
                last_rebalance_time = current_time
                cache.set('last_rebalance_time', last_rebalance_time)
                print(f'last_rebalance_time updated to: {last_rebalance_time}')
                return True
            else:
                print("Rebalancing is not required at this time.")
                return False
        else:
            print("Rebalancing is not required at this time.")
            return False


In [81]:
rebalance_frequency

360

In [82]:
from python_scripts.utils import set_global_seed

In [83]:
def run_sim(model,prices_df, hist_comp, seed, rebalancing_frequency):
    print(f'prices before sim: {prices_df}')

    model = PPO.load(f"AI_Models/{model}")
    
    env = Portfolio(df=prices_df, compositions=hist_comp, seed=seed, rebalance_frequency=rebalancing_frequency,
                    risk_free_annual=current_risk_free)

    set_global_seed(env,seed)

    states = []
    rewards = []
    actions = []
    portfolio_values = []
    compositions = []
    dates = []

    # Reset the environment to get the initial state
    state, _ = env.reset(seed=seed)  # Get the initial state
    done = False

    while not done:
        # Use the model to predict the action
        action, _states = model.predict(state)
        
        # Take a step in the environment
        next_state, reward, done, truncated, info = env.step(action)
        
        # Break the loop if done to avoid processing a None state
        if done:
            print("Episode done. Exiting the loop.")
            break
        
        # Normalize the action to ensure it sums to 1
        # action = action / np.sum(action)
        
        # Store the results
        if next_state is not None:
            states.append(next_state.flatten())  # Ensure the state is flattened
        rewards.append(reward)
        actions.append(action.flatten())  # Ensure the action is flattened
        compositions.append(env.portfolio)  # Store the portfolio composition
        print(f'Action: {action}')

        # Update the state
        state = next_state

        # Print debug information
        print(f"Step: {env.current_step}")
        print(f"State: {next_state}")
        print(f'Action: {action}')
        print(f"Reward: {reward}")
        print(f"Done: {done}")
        print(f"Info: {info}")

    states_df = env.get_states_df()
    rewards_df = env.get_rewards_df()
    actions_df = env.get_actions_df()
    returns_df = env.get_returns_df()
    sortino_ratios = env.get_sortino_ratios_df()


    return states_df, rewards_df, actions_df, returns_df, sortino_ratios



In [84]:
formatted_today_utc

'2024-12-16 23:00:00'

In [85]:
hist_comp = historical_data.copy()
hist_comp.set_index('date', inplace=True)
hist_comp.index = pd.to_datetime(hist_comp.index)
# hist_comp = prepare_data_for_simulation(hist_comp, start_date, end_date)
print(f'hist comp for env {hist_comp}')

hist comp for env                          CRV comp  LINK comp  UNI comp  WBTC comp
date                                                             
2024-12-16 07:00:00  9.239875e-11   0.940108  0.059892        0.0
2024-12-16 22:00:00  9.143182e-11   0.941771  0.058229        0.0
2024-12-16 23:00:00  9.143182e-11   0.941771  0.058229        0.0


In [86]:
hist_comp

,CRV comp,LINK comp,UNI comp,WBTC comp
date,,,,
2024-12-16 07:00:00,9.239875e-11,0.940108,0.059892,0.0
2024-12-16 22:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-16 23:00:00,9.143182e-11,0.941771,0.058229,0.0


In [87]:
start_date

'2024-12-16 23:00:00'

In [88]:
oralce_copy = oracle_prices.copy().set_index('hour')

In [89]:
hist_comp

,CRV comp,LINK comp,UNI comp,WBTC comp
date,,,,
2024-12-16 07:00:00,9.239875e-11,0.940108,0.059892,0.0
2024-12-16 22:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-16 23:00:00,9.143182e-11,0.941771,0.058229,0.0


In [90]:
classifier_data

,symbol,token_address,latest_hour,latest_price,sixty_day_price,sixty_d_return,rolling_7d_avg,rolling_30d_avg,volume,average_order,__row_index
0,CRV,0x11cdb42b0eb46d95f990bedd4695a6e3fa034978,2024-12-16 02:00:00+00:00,1.15,0.29075,2.955288,1.129338,0.752708,5.206880e+07,625.902174,0
1,LINK,0xf97f4df75117a78c1a5a0dbb814af92458539fb4,2024-12-16 02:00:00+00:00,29.34,13.84000,1.119942,26.477811,20.614286,1.569272e+08,1008.179565,1
2,UNI,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,2024-12-16 02:00:00+00:00,17.31,8.54000,1.026932,16.874615,13.356671,2.370339e+07,538.799173,2
9,WBTC,0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f,2024-12-16 02:00:00+00:00,105146.00,90781.00000,0.158238,99561.781065,96637.266297,2.611187e+09,2823.725555,9


In [91]:
formatted_today_utc

'2024-12-16 23:00:00'

In [92]:
oralce_copy.index

DatetimeIndex(['2024-12-16 07:00:00', '2024-12-16 22:00:00',
                               'NaT', '2024-12-16 23:00:00',
               '2024-12-17 00:00:00', '2024-12-17 01:00:00',
               '2024-12-17 02:00:00', '2024-12-17 03:00:00',
               '2024-12-17 04:00:00'],
              dtype='datetime64[ns]', name='hour', freq=None)

In [93]:
oracle_prices

,hour,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
0,2024-12-16 07:00:00,104384.0,16.70,29.04,1.099
1,2024-12-16 22:00:00,105760.0,16.60,29.66,1.099
2,NaT,105641.0,15.99,28.65,1.071
3,2024-12-16 23:00:00,105775.0,16.55,29.08,1.092
4,2024-12-17 00:00:00,105602.0,16.45,29.08,1.082
5,2024-12-17 01:00:00,105600.0,16.29,28.84,1.073
6,2024-12-17 02:00:00,105641.0,15.99,28.65,1.071
7,2024-12-17 03:00:00,105641.0,15.99,28.65,1.071
8,2024-12-17 04:00:00,105641.0,15.99,28.65,1.071


In [94]:
oralce_copy

,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
hour,,,,
2024-12-16 07:00:00,104384.0,16.70,29.04,1.099
2024-12-16 22:00:00,105760.0,16.60,29.66,1.099
NaT,105641.0,15.99,28.65,1.071
2024-12-16 23:00:00,105775.0,16.55,29.08,1.092
2024-12-17 00:00:00,105602.0,16.45,29.08,1.082
2024-12-17 01:00:00,105600.0,16.29,28.84,1.073
2024-12-17 02:00:00,105641.0,15.99,28.65,1.071
2024-12-17 03:00:00,105641.0,15.99,28.65,1.071
2024-12-17 04:00:00,105641.0,15.99,28.65,1.071


In [95]:
hist_comp

,CRV comp,LINK comp,UNI comp,WBTC comp
date,,,,
2024-12-16 07:00:00,9.239875e-11,0.940108,0.059892,0.0
2024-12-16 22:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-16 23:00:00,9.143182e-11,0.941771,0.058229,0.0


In [ ]:
# hist_comp.set_index('hour',inplace=True)
# hist_comp.index = pd.to_datetime(hist_comp.index)

KeyError: "None of ['hour'] are in the columns"

In [94]:
# hist_comp.index = pd.to_datetime(hist_comp.index)

In [95]:
hist_comp

,CRV comp,LINK comp,UNI comp,WBTC comp
hour,,,,
2024-12-16 07:00:00,9.239875e-11,0.940108,0.059892,0.0
2024-12-16 22:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-16 23:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 00:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 01:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 02:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 03:00:00,9.143182e-11,0.941771,0.058229,0.0


In [96]:
oralce_copy

,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
hour,,,,
2024-12-16 07:00:00,104384.0,16.70,29.04,1.099
2024-12-16 22:00:00,105760.0,16.60,29.66,1.099
2024-12-16 23:00:00,105775.0,16.55,29.08,1.092
2024-12-17 00:00:00,105602.0,16.45,29.08,1.082
2024-12-17 01:00:00,105600.0,16.29,28.84,1.073
2024-12-17 02:00:00,105641.0,15.99,28.65,1.071


In [ ]:
states_df, rewards_df, actions_df, portfolio_values_df, composition = run_sim(
    model_name,oralce_copy,hist_comp,seed,rebalance_frequency
)


prices before sim:                      WBTC_Price  UNI_Price  LINK_Price  CRV_Price
hour                                                             
2024-12-16 07:00:00    104384.0      16.70       29.04      1.099
2024-12-16 22:00:00    105760.0      16.60       29.66      1.099
2024-12-16 23:00:00    105775.0      16.55       29.08      1.092
2024-12-17 00:00:00    105602.0      16.45       29.08      1.082
2024-12-17 01:00:00    105600.0      16.29       28.84      1.073
2024-12-17 02:00:00    105641.0      15.99       28.65      1.071
seed: 20

--- Step 0 ---
Action after clipping: [0.         0.06277151 1.         0.        ]
Current prices: [1.04384e+05 1.67000e+01 2.90400e+01 1.09900e+00]
Portfolio:                          CRV comp  LINK comp  UNI comp  WBTC comp
hour                                                             
2024-12-16 07:00:00  9.239875e-11   0.940108  0.059892        0.0
2024-12-16 22:00:00  9.143182e-11   0.941771  0.058229        0.0
2024-12-16 23:00:0

e:\Projects\steth_optimizer\steth_optimizer\Lib\site-packages\scipy\optimize\_differentiable_functions.py:231: UserWarning:

delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.

e:\Projects\portfolio_optimizers\dao_advisor_project\models\testnet_model.py:439: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

e:\Projects\portfolio_optimizers\dao_advisor_project\models\testnet_model.py:439: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

e:\Projects\portfolio_optimizers\dao_advis

tokens_price:['WBTC', 'UNI', 'LINK', 'CRV']
tokens_comp:['CRV', 'LINK', 'UNI', 'WBTC']
common_tokens: {'WBTC', 'CRV', 'UNI', 'LINK'}
comp_columns: ['WBTC comp', 'CRV comp', 'UNI comp', 'LINK comp']
common_index: DatetimeIndex(['2024-12-16 07:00:00', '2024-12-16 22:00:00',
               '2024-12-16 23:00:00'],
              dtype='datetime64[ns]', name='hour', freq=None)
compositions:                      WBTC comp      CRV comp  UNI comp  LINK comp
hour                                                             
2024-12-16 07:00:00        0.0  9.239875e-11  0.059892   0.940108
2024-12-16 22:00:00        0.0  9.143182e-11  0.058229   0.941771
2024-12-16 23:00:00        0.0  9.143182e-11  0.058229   0.941771
returns:                     WBTC_Price  CRV_Price  UNI_Price  LINK_Price
hour                                                             
2024-12-16 07:00:00    0.000000    0.00000   0.000000    0.000000
2024-12-16 22:00:00    0.013096    0.00000  -0.006006    0.021125
2024-12-16

e:\Projects\steth_optimizer\steth_optimizer\Lib\site-packages\scipy\optimize\_differentiable_functions.py:231: UserWarning:

delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.



`xtol` termination condition is satisfied.
Number of iterations: 349, function evaluations: 3240, CG iterations: 389, optimality: 1.17e-02, constraint violation: 0.00e+00, execution time: 0.84 s.
portfolio:                          CRV comp  LINK comp  UNI comp  WBTC comp
hour                                                             
2024-12-16 07:00:00  9.239875e-11   0.940108  0.059892        0.0
2024-12-16 22:00:00  9.143182e-11   0.941771  0.058229        0.0
2024-12-16 23:00:00  9.143182e-11   0.941771  0.058229        0.0
2024-12-17 00:00:00  9.143182e-11   0.941771  0.058229        0.0
2024-12-17 01:00:00  9.143182e-11   0.941771  0.058229        0.0
2024-12-17 02:00:00  9.143182e-11   0.941771  0.058229        0.0
2024-12-17 03:00:00  9.143182e-11   0.941771  0.058229        0.0, optimized_portfolio: [1.00000000e+00 5.63984256e-17 1.00000000e-02 5.29061670e-16], sortino_ratio: 0.08042310587504761
self.portfolio.iloc[-1]: CRV comp     9.143182e-11
LINK comp    9.417713e-01
UN

e:\Projects\portfolio_optimizers\dao_advisor_project\models\testnet_model.py:439: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

e:\Projects\portfolio_optimizers\dao_advisor_project\models\testnet_model.py:439: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [98]:
portfolio_values_df

,Return,Date
0,0.000000,2024-12-16 07:00:00
1,0.019545,2024-12-16 22:00:00
2,-0.018774,2024-12-16 23:00:00
3,-0.000353,2024-12-17 00:00:00
4,-0.008374,2024-12-17 01:00:00


In [99]:
update_model_actions(actions_df.to_dict())

current_time = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00')
current_time = pd.to_datetime(current_time)

model actions before update: Empty DataFrame
Columns: []
Index: []
new data:    WBTC_weight  UNI_weight  LINK_weight  CRV_weight                Date
0          0.0    0.059064     0.940936         0.0 2024-12-16 07:00:00


In [100]:
new_compositions = {
            token: float(model_actions.iloc[-1][f"{token}_weight"]) for token in TOKEN_CONTRACTS
                }

In [101]:
new_compositions

{'CRV': 0.0,
 'LINK': 0.9409359693527222,
 'UNI': 0.059063974767923355,
 'WBTC': 0.0}

In [102]:
print(last_rebalance_time)

None


In [103]:
latest_comp = hist_comp.iloc[-1]

latest_comp.index = latest_comp.index.str.replace('comp','')
latest_comp = latest_comp.to_dict()

In [104]:
latest_comp

{'CRV ': 9.143181780762791e-11,
 'LINK ': 0.9417713132046657,
 'UNI ': 0.05822868670390256,
 'WBTC ': 0.0}

In [105]:
new_compositions

{'CRV': 0.0,
 'LINK': 0.9409359693527222,
 'UNI': 0.059063974767923355,
 'WBTC': 0.0}

In [106]:
def composition_difference_exceeds_threshold(latest_comp, new_compositions, threshold=0.05):
    """
    Check if the difference between the latest and new compositions exceeds the threshold.

    Parameters:
    - latest_comp: Dict of the current portfolio composition.
    - new_compositions: Dict of the target portfolio composition.
    - threshold: The tolerance threshold (default 5%).

    Returns:
    - True if the difference exceeds the threshold for any token, False otherwise.
    """
    print(f'new_compositions: {new_compositions}')
    print(f'latest_comp: {latest_comp}')
    for token in new_compositions:
        latest_value = latest_comp.get(token, 0)
        new_value = new_compositions.get(token, 0)
        
        # Calculate the absolute difference
        difference = abs(latest_value - new_value)
        
        print(f"{token}: Latest: {latest_value:.6f}, New: {new_value:.6f}, Difference: {difference:.6f}")
        
        # Check if the difference exceeds the threshold
        if difference > threshold:
            return True

    return False

In [107]:
# target_compositions = {
#     'WBTC': 0.25,
#     'LINK': 0.25,
#     'UNI': 0.25,
#     'CRV': 0.25
# }

if should_rebalance(start_date,actions_df,rebalance_frequency):
    if composition_difference_exceeds_threshold(latest_comp, new_compositions, diff_tolerance_threshold):
        rebalance_portfolio(uniswap, 
                            TOKEN_CONTRACTS, 
                            TOKEN_DECIMALS, 
                            new_compositions, 
                            ACCOUNT_ADDRESS, 
                        )
        

last rebal time: None
current time 2024-12-16 22:00:00
actions df    WBTC_weight  UNI_weight  LINK_weight  CRV_weight                Date
0          0.0    0.059064     0.940936         0.0 2024-12-16 07:00:00
rebalancing frequency 360
should rebal current time: 2024-12-16 22:00:00
actions df:    WBTC_weight  UNI_weight  LINK_weight  CRV_weight                Date
0          0.0    0.059064     0.940936         0.0 2024-12-16 07:00:00
should rebal last rebal time: 2024-12-16 07:00:00
hours since last rebal: 15.0
Rebalancing is not required at this time.


In [108]:
historical_data

,CRV comp,LINK comp,UNI comp,WBTC comp,date
0,9.239875e-11,0.940108,0.059892,0.0,2024-12-16 07:00:00
1,9.143182e-11,0.941771,0.058229,0.0,2024-12-16 22:00:00


In [109]:
oracle_prices

,hour,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
0,2024-12-16 07:00:00,104384.0,16.70,29.04,1.099
1,2024-12-16 22:00:00,105760.0,16.60,29.66,1.099
2,2024-12-16 23:00:00,105775.0,16.55,29.08,1.092
3,2024-12-17 00:00:00,105602.0,16.45,29.08,1.082
4,2024-12-17 01:00:00,105600.0,16.29,28.84,1.073
5,2024-12-17 02:00:00,105641.0,15.99,28.65,1.071


In [110]:
def calculate_log_returns(prices):
    return np.log(prices / prices.shift(1)).fillna(0)

In [111]:
price_returns = calculate_log_returns(oralce_copy)
price_returns

,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
hour,,,,
2024-12-16 07:00:00,0.000000,0.000000,0.000000,0.000000
2024-12-16 22:00:00,0.013096,-0.006006,0.021125,0.000000
2024-12-16 23:00:00,0.000142,-0.003017,-0.019749,-0.006390
2024-12-17 00:00:00,-0.001637,-0.006061,0.000000,-0.009200
2024-12-17 01:00:00,-0.000019,-0.009774,-0.008287,-0.008353
2024-12-17 02:00:00,0.000388,-0.018588,-0.006610,-0.001866


In [112]:
hist_comp.index = pd.to_datetime(hist_comp.index)

In [113]:
price_returns.index

DatetimeIndex(['2024-12-16 07:00:00', '2024-12-16 22:00:00',
               '2024-12-16 23:00:00', '2024-12-17 00:00:00',
               '2024-12-17 01:00:00', '2024-12-17 02:00:00'],
              dtype='datetime64[ns]', name='hour', freq=None)

In [114]:
oralce_copy

,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
hour,,,,
2024-12-16 07:00:00,104384.0,16.70,29.04,1.099
2024-12-16 22:00:00,105760.0,16.60,29.66,1.099
2024-12-16 23:00:00,105775.0,16.55,29.08,1.092
2024-12-17 00:00:00,105602.0,16.45,29.08,1.082
2024-12-17 01:00:00,105600.0,16.29,28.84,1.073
2024-12-17 02:00:00,105641.0,15.99,28.65,1.071


In [115]:
hist_comp

,CRV comp,LINK comp,UNI comp,WBTC comp
hour,,,,
2024-12-16 07:00:00,9.239875e-11,0.940108,0.059892,0.0
2024-12-16 22:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-16 23:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 00:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 01:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 02:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 03:00:00,9.143182e-11,0.941771,0.058229,0.0


In [116]:
current_risk_free

0.044199999999999996

In [117]:
from scipy.optimize import minimize, Bounds, LinearConstraint

In [118]:
oralce_copy

,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
hour,,,,
2024-12-16 07:00:00,104384.0,16.70,29.04,1.099
2024-12-16 22:00:00,105760.0,16.60,29.66,1.099
2024-12-16 23:00:00,105775.0,16.55,29.08,1.092
2024-12-17 00:00:00,105602.0,16.45,29.08,1.082
2024-12-17 01:00:00,105600.0,16.29,28.84,1.073
2024-12-17 02:00:00,105641.0,15.99,28.65,1.071


In [119]:
hist_comp

,CRV comp,LINK comp,UNI comp,WBTC comp
hour,,,,
2024-12-16 07:00:00,9.239875e-11,0.940108,0.059892,0.0
2024-12-16 22:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-16 23:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 00:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 01:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 02:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 03:00:00,9.143182e-11,0.941771,0.058229,0.0


In [120]:
current_risk_free

0.044199999999999996

In [121]:
oracle_copy_2 = oracle_prices.copy()

In [122]:
oracle_copy_2

,hour,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
0,2024-12-16 07:00:00,104384.0,16.70,29.04,1.099
1,2024-12-16 22:00:00,105760.0,16.60,29.66,1.099
2,2024-12-16 23:00:00,105775.0,16.55,29.08,1.092
3,2024-12-17 00:00:00,105602.0,16.45,29.08,1.082
4,2024-12-17 01:00:00,105600.0,16.29,28.84,1.073
5,2024-12-17 02:00:00,105641.0,15.99,28.65,1.071


In [123]:
oracle_copy_2.set_index('hour',inplace=True)
oracle_copy_2.index = pd.to_datetime(oracle_copy_2.index)

In [124]:
oracle_copy_2.index

DatetimeIndex(['2024-12-16 07:00:00', '2024-12-16 22:00:00',
               '2024-12-16 23:00:00', '2024-12-17 00:00:00',
               '2024-12-17 01:00:00', '2024-12-17 02:00:00'],
              dtype='datetime64[ns]', name='hour', freq=None)

In [125]:
hist_comp.index

DatetimeIndex(['2024-12-16 07:00:00', '2024-12-16 22:00:00',
               '2024-12-16 23:00:00', '2024-12-17 00:00:00',
               '2024-12-17 01:00:00', '2024-12-17 02:00:00',
               '2024-12-17 03:00:00'],
              dtype='datetime64[ns]', name='hour', freq=None)

In [126]:
# hist_comp.columns = hist_comp.columns.str.replace('comp','')

In [127]:
hist_comp

,CRV comp,LINK comp,UNI comp,WBTC comp
hour,,,,
2024-12-16 07:00:00,9.239875e-11,0.940108,0.059892,0.0
2024-12-16 22:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-16 23:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 00:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 01:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 02:00:00,9.143182e-11,0.941771,0.058229,0.0
2024-12-17 03:00:00,9.143182e-11,0.941771,0.058229,0.0


In [128]:
oracle_copy_2

,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
hour,,,,
2024-12-16 07:00:00,104384.0,16.70,29.04,1.099
2024-12-16 22:00:00,105760.0,16.60,29.66,1.099
2024-12-16 23:00:00,105775.0,16.55,29.08,1.092
2024-12-17 00:00:00,105602.0,16.45,29.08,1.082
2024-12-17 01:00:00,105600.0,16.29,28.84,1.073
2024-12-17 02:00:00,105641.0,15.99,28.65,1.071


In [129]:
optimized_weights, returns, sortino_ratio = mvo(oracle_copy_2, hist_comp, current_risk_free)


Converted hourly risk-free rate: 4.93734574957827e-06
data at mvo:
                     WBTC_Price  UNI_Price  LINK_Price  CRV_Price
hour                                                             
2024-12-16 07:00:00    104384.0      16.70       29.04      1.099
2024-12-16 22:00:00    105760.0      16.60       29.66      1.099
2024-12-16 23:00:00    105775.0      16.55       29.08      1.092
2024-12-17 00:00:00    105602.0      16.45       29.08      1.082
2024-12-17 01:00:00    105600.0      16.29       28.84      1.073
2024-12-17 02:00:00    105641.0      15.99       28.65      1.071
portfolio:                          CRV comp  LINK comp  UNI comp  WBTC comp
hour                                                             
2024-12-16 07:00:00  9.239875e-11   0.940108  0.059892        0.0
2024-12-16 22:00:00  9.143182e-11   0.941771  0.058229        0.0
2024-12-16 23:00:00  9.143182e-11   0.941771  0.058229        0.0
2024-12-17 00:00:00  9.143182e-11   0.941771  0.058229        0.

In [130]:
sortino_ratio

-1.198968264422309

In [131]:
optimized_weights

array([4.35443413e-05, 9.99967702e-01, 4.40397738e-04, 9.46764644e-03])

In [132]:
returns

hour
2024-12-16 07:00:00    0.000000
2024-12-16 22:00:00    0.019545
2024-12-16 23:00:00   -0.018774
2024-12-17 00:00:00   -0.000353
2024-12-17 01:00:00   -0.008374
2024-12-17 02:00:00   -0.007307
Name: Portfolio_Return, dtype: float64

In [133]:
normalized_portfolio_value = (1 + returns).cumprod()
normalized_portfolio_value

hour
2024-12-16 07:00:00    1.000000
2024-12-16 22:00:00    1.019545
2024-12-16 23:00:00    1.000404
2024-12-17 00:00:00    1.000051
2024-12-17 01:00:00    0.991677
2024-12-17 02:00:00    0.984430
Name: Portfolio_Return, dtype: float64

In [134]:
normalized_prices = (1 + price_returns).cumprod()
normalized_prices


,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
hour,,,,
2024-12-16 07:00:00,1.000000,1.000000,1.000000,1.000000
2024-12-16 22:00:00,1.013096,0.993994,1.021125,1.000000
2024-12-16 23:00:00,1.013240,0.990996,1.000959,0.993610
2024-12-17 00:00:00,1.011581,0.984989,1.000959,0.984469
2024-12-17 01:00:00,1.011562,0.975362,0.992664,0.976246
2024-12-17 02:00:00,1.011955,0.957232,0.986103,0.974425


In [135]:
price_returns

,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
hour,,,,
2024-12-16 07:00:00,0.000000,0.000000,0.000000,0.000000
2024-12-16 22:00:00,0.013096,-0.006006,0.021125,0.000000
2024-12-16 23:00:00,0.000142,-0.003017,-0.019749,-0.006390
2024-12-17 00:00:00,-0.001637,-0.006061,0.000000,-0.009200
2024-12-17 01:00:00,-0.000019,-0.009774,-0.008287,-0.008353
2024-12-17 02:00:00,0.000388,-0.018588,-0.006610,-0.001866


In [136]:
def normalize(data, date):
    # Convert Series to DataFrame and ensure it has a 'cumulative_return' column
    if isinstance(data, pd.Series):
        data = data.to_frame(name='cumulative_return')
    else:
        data = data.rename(columns={'Portfolio Return': 'cumulative_return'})

    # Convert the DataFrame to reset index to allow slicing by position
    cumulative_return_df = data.reset_index()
    print('cum df', cumulative_return_df)

    # Rename the index to 'DAY' if it doesn't already have that column
    if 'index' in cumulative_return_df.columns:
        cumulative_return_df = cumulative_return_df.rename(columns={'index': 'DAY'})

    # Check for and handle duplicate 'DAY' columns
    if cumulative_return_df.columns.duplicated().any():
        cumulative_return_df = cumulative_return_df.loc[:, ~cumulative_return_df.columns.duplicated()]

    # Convert the 'DAY' column to datetime if it's not already
    cumulative_return_df['DAY'] = pd.to_datetime(cumulative_return_df['DAY'])

    # Filter the DataFrame to start from the specified date
    cumulative_return_df = cumulative_return_df[cumulative_return_df['DAY'] >= pd.to_datetime(date)]

    # Prepare base return (use cumulative_return as the base)
    base_return = cumulative_return_df.copy().dropna()
    print('base return', base_return)

    # Drop any existing columns and rename them
    base_return = base_return[['DAY', 'cumulative_return']].rename(columns={'cumulative_return': 'base_cumulative_return'})

    # Combine results
    combined = base_return.sort_values('DAY')

    # Check if combined is empty
    if combined.empty:
        print("Warning: No data available after the specified start date.")
        # Optionally, return an empty DataFrame or handle as needed
        return pd.DataFrame(columns=['DAY', 'treasury_return']).set_index('DAY')

    # Normalize returns
    first_value = combined['base_cumulative_return'].iloc[0]  # Get the first value
    combined['treasury_return'] = 100 + (100 * (combined['base_cumulative_return'] - first_value))

    # Final output
    normalized_returns = combined[['DAY', 'treasury_return']]
    normalized_returns.set_index('DAY', inplace=True)

    return normalized_returns

In [137]:
current_risk_free

0.044199999999999996

In [138]:
sortino_ratio

-1.198968264422309

In [139]:
def normalize_log_returns(data, date):
    # Convert the DataFrame to use 'DAY' as the index

    # Calculate log returns
    log_returns = np.log(data / data.shift(1)).dropna()

    # Calculate cumulative log returns
    cumulative_log_returns = log_returns.cumsum()

    # Convert cumulative log returns to cumulative simple returns
    cumulative_simple_returns = np.exp(cumulative_log_returns) - 1

    # Convert the DataFrame to reset index to allow slicing by position
    cumulative_return_df = cumulative_simple_returns.reset_index()
    
    return log_returns, cumulative_log_returns, cumulative_simple_returns

# Visualizations

In [140]:
from python_scripts.utils import normalize_asset_returns

In [ ]:
model_balances = get_balance(TOKEN_CONTRACTS,TOKEN_DECIMALS,ACCOUNT_ADDRESS)

model_balances_usd = convert_to_usd(model_balances,latest_prices,TOKEN_CONTRACTS)
portfolio_balance = sum(model_balances_usd.values())



Balances for account 0x75baD5ae9f46e8AEBf61e4A7179cEf3A0CeD6766: {'CRV': 4.291008269e-09, 'LINK': 1.652236831399549, 'UNI': 0.18303746793932313, 'WBTC': 0.0}
balances: dict_keys(['CRV', 'LINK', 'UNI', 'WBTC'])
TOKEN_CONTRACTS.keys(): dict_keys(['CRV', 'LINK', 'UNI', 'WBTC'])


In [142]:
portfolio_dict = {
    "Portfolio Value": portfolio_balance,
    "date": end_date
}

# Save new portfolio to CSV

update_portfolio_data(portfolio_dict)

values: {'Portfolio Value': 50.26335433654252, 'date': '2024-12-17 03:00:00'}


In [143]:
comp_dict_data = calculate_compositions(model_balances_usd, portfolio_balance)

print(f'comp_dict_data: {comp_dict_data}')

comp_dict = {**comp_dict_data, 'date': formatted_today_utc}

update_historical_data(comp_dict)

comp_dict_data: {'CRV comp': 9.143181780762791e-11, 'LINK comp': 0.9417713132046657, 'UNI comp': 0.05822868670390256, 'WBTC comp': 0.0}


In [144]:
portfolio_values_df

,Return,Date
0,0.000000,2024-12-16 07:00:00
1,0.019545,2024-12-16 22:00:00
2,-0.018774,2024-12-16 23:00:00
3,-0.000353,2024-12-17 00:00:00
4,-0.008374,2024-12-17 01:00:00


In [145]:
# portfolio_values_df.set_index('Date', inplace=True)
# portfolio_return = calculate_cumulative_return(portfolio_values_df, 'Portfolio_Value')

# prices_df.index = pd.to_datetime(prices_df.index)
print(f'price index: {prices_df.index}')

# Assuming 'portfolio_values_df' is your DataFrame with a timezone-aware index
print(f'portfolio index: {portfolio_values_df.index}')
# portfolio_values_df.index = portfolio_values_df.index.tz_localize(None)

print(f'portfolio_values_df: {portfolio_values_df}')
print(f'first portfolio_values_df: {portfolio_values_df.values[0]}')
viz_port_values = portfolio_values_df.copy()
# viz_port_values.set_index('Date',inplace=True)

print(F'viz_port_values: {viz_port_values}')

viz_port_values.set_index('Date',inplace=True)

price index: RangeIndex(start=0, stop=5, step=1)
portfolio index: RangeIndex(start=0, stop=5, step=1)
portfolio_values_df:      Return                Date
0  0.000000 2024-12-16 07:00:00
1  0.019545 2024-12-16 22:00:00
2 -0.018774 2024-12-16 23:00:00
3 -0.000353 2024-12-17 00:00:00
4 -0.008374 2024-12-17 01:00:00
first portfolio_values_df: [0.0 Timestamp('2024-12-16 07:00:00')]
viz_port_values:      Return                Date
0  0.000000 2024-12-16 07:00:00
1  0.019545 2024-12-16 22:00:00
2 -0.018774 2024-12-16 23:00:00
3 -0.000353 2024-12-17 00:00:00
4 -0.008374 2024-12-17 01:00:00


In [ ]:
# viz_port_values = prepare_data_for_simulation(viz_port_values,formatted_today_utc,end_date)


price index: RangeIndex(start=0, stop=7, step=1)


AttributeError: 'RangeIndex' object has no attribute 'tz_localize'

In [149]:
viz_port_values.set_index('hour',inplace=True)


In [157]:
viz_port_values

,Return
hour,
2024-12-16 07:00:00,0.000000
2024-12-16 22:00:00,0.019545
2024-12-16 23:00:00,-0.018774
2024-12-17 00:00:00,-0.000353
2024-12-17 01:00:00,-0.008374
2024-12-17 02:00:00,-0.008374
2024-12-17 03:00:00,-0.008374


In [159]:
import pandas as pd
import numpy as np

def normalize_log_returns(log_returns_timeseries, start_date, end_date, normalize_value=1e4):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filter the data based on the start date and end date
    filtered_data = log_returns_timeseries[(log_returns_timeseries.index >= start_date) & 
                                           (log_returns_timeseries.index <= end_date)].copy()
    
    if filtered_data.empty:
        print("Filtered data is empty after applying the date filter.")
        return pd.DataFrame()

    # Initialize a dictionary to store normalized values for each asset (column)
    normalized_values = {col: [normalize_value] for col in filtered_data.columns}
    dates = [filtered_data.index[0]]  # Use the original start date for labeling

    # Loop through the filtered data and calculate normalized returns for each asset
    for i in range(len(filtered_data)):
        for col in filtered_data.columns:
            log_return = filtered_data[col].iloc[i]
            normalized_values[col].append(normalized_values[col][-1] * np.exp(log_return))

        dates.append(filtered_data.index[i])

    # Create a DataFrame with normalized values for each asset, excluding the initial date
    normalized_returns_df = pd.DataFrame({col: vals[1:] for col, vals in normalized_values.items()}, index=dates[1:])

    return normalized_returns_df


In [160]:
norm_port = normalize_log_returns(viz_port_values, viz_port_values.index.min(),viz_port_values.index.max(), 100)
norm_port

,Return
2024-12-16 07:00:00,100.000000
2024-12-16 22:00:00,101.973759
2024-12-16 23:00:00,100.077124
2024-12-17 00:00:00,100.041812
2024-12-17 01:00:00,99.207569
2024-12-17 02:00:00,98.380282
2024-12-17 03:00:00,97.559895


In [ ]:
# oralce_copy = prepare_data_for_simulation(oralce_copy, start_date, end_date)


price index: RangeIndex(start=0, stop=7, step=1)


AttributeError: 'RangeIndex' object has no attribute 'tz_localize'

In [ ]:
oralce_copy.set_index('hour',inplace=True)

In [176]:
norm_prices = normalize_asset_returns(oralce_copy, viz_port_values.index.min(),viz_port_values.index.max(), 100)
norm_prices

,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
2024-12-16 07:00:00,100.000000,100.000000,100.000000,100.000000
2024-12-16 22:00:00,101.318210,99.401198,102.134986,100.000000
2024-12-16 23:00:00,101.332580,99.101796,100.137741,99.363057
2024-12-17 00:00:00,101.166845,98.502994,100.137741,98.453139
2024-12-17 01:00:00,101.164929,97.544910,99.311295,97.634213
2024-12-17 02:00:00,101.204208,95.748503,98.657025,97.452229
2024-12-17 03:00:00,101.204208,95.748503,98.657025,97.452229


In [177]:
norm_prices

,WBTC_Price,UNI_Price,LINK_Price,CRV_Price
2024-12-16 07:00:00,100.000000,100.000000,100.000000,100.000000
2024-12-16 22:00:00,101.318210,99.401198,102.134986,100.000000
2024-12-16 23:00:00,101.332580,99.101796,100.137741,99.363057
2024-12-17 00:00:00,101.166845,98.502994,100.137741,98.453139
2024-12-17 01:00:00,101.164929,97.544910,99.311295,97.634213
2024-12-17 02:00:00,101.204208,95.748503,98.657025,97.452229
2024-12-17 03:00:00,101.204208,95.748503,98.657025,97.452229


In [178]:
norm_port

,Return
2024-12-16 07:00:00,100.000000
2024-12-16 22:00:00,101.973759
2024-12-16 23:00:00,100.077124
2024-12-17 00:00:00,100.041812
2024-12-17 01:00:00,99.207569
2024-12-17 02:00:00,98.380282
2024-12-17 03:00:00,97.559895


In [ ]:
norm_port.rename(columns={'Portfolio_Value':f'{model_name} Portfolio Value'},inplace=True)

nom_comp = pd.merge(
    norm_port, norm_prices, left_index=True, right_index=True, how='left'
)

print(f'historical_data: {historical_data}')
print(f'norm_port: {norm_port}')

plot_historical_data = historical_data.copy()
plot_historical_data.set_index('date',inplace=True)
plot_historical_data.index = pd.to_datetime(plot_historical_data.index)
plot_historical_data = plot_historical_data.resample('H').ffill().bfill()
print(f'plot_historical_data: {plot_historical_data.index}')

model_fig1 = visualization_pipeline(
    df=nom_comp,
    title='model_normalized',
    chart_type = 'line',
    cols_to_plot='All',
    tickprefix=dict(y1='$',y2=None),
    show_legend=True,
    decimals=True,
    sort_list = False,
    tickformat=dict(x='%b %d <br> %y',y1=None,y2=None),
    legend_placement=dict(x=0.05,y=0.8),
    font_family=font_family
)

chartBuilder(
    fig = model_fig1,
    show=True,
    save=False,
    title='Normalized Model Performance',
    subtitle=f'{model_name} Portfolio',
)

model_fig2 = visualization_pipeline(
    df=plot_historical_data,
    title='model_historical',
    chart_type='bar',
    to_percent=True,
    show_legend=True,
    sort_list = False,
    legend_placement=dict(x=0.1,y=1.3),
    cols_to_plot='All',
    buffer=1,
    ticksuffix=dict(y1='%',y2=None),
    margin=dict(t=150,b=0,l=0,r=0),
    font_family=font_family
)

chartBuilder(
    fig=model_fig2,
    title='Portfolio Composition Over Time',
    date_xy=dict(x=0.1,y=1.4),
    show=True,
    save=False
)

aum_df = historical_port_values.copy()
aum_df.set_index('date',inplace=True)
aum_df.index = pd.to_datetime(aum_df.index)

print(f'aum_df: {aum_df}')

model_fig3 = visualization_pipeline(
    df=aum_df,
    title='viz_port_values',
    chart_type='line',
    area=True,
    show_legend=True,
    sort_list = False,
    legend_placement=dict(x=0.1,y=1.3),
    cols_to_plot='All',
    tickprefix=dict(y1='$',y2=None),
    margin=dict(t=150,b=0,l=0,r=0),
    font_family=font_family
)

chartBuilder(
    fig=model_fig3,
    title='AUM Over Time',
    date_xy=dict(x=0.1,y=1.4),
    show=True,
    save=False
)


historical_data:        CRV comp  LINK comp  UNI comp  WBTC comp                 date
0  9.239875e-11   0.940108  0.059892        0.0  2024-12-16 07:00:00
2  9.143182e-11   0.941771  0.058229        0.0  2024-12-16 22:00:00
norm_port:                          Return
2024-12-16 07:00:00  100.000000
2024-12-16 22:00:00  101.973759
2024-12-16 23:00:00  100.077124
2024-12-17 00:00:00  100.041812
2024-12-17 01:00:00   99.207569
2024-12-17 02:00:00   98.380282
2024-12-17 03:00:00   97.559895
plot_historical_data: DatetimeIndex(['2024-12-16 07:00:00', '2024-12-16 08:00:00',
               '2024-12-16 09:00:00', '2024-12-16 10:00:00',
               '2024-12-16 11:00:00', '2024-12-16 12:00:00',
               '2024-12-16 13:00:00', '2024-12-16 14:00:00',
               '2024-12-16 15:00:00', '2024-12-16 16:00:00',
               '2024-12-16 17:00:00', '2024-12-16 18:00:00',
               '2024-12-16 19:00:00', '2024-12-16 20:00:00',
               '2024-12-16 21:00:00', '2024-12-16 22:00:00']

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_37080\4193398726.py:13: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: 2024-12-16 07:00:00
turn_to_time: True
save:False
logo path: ../img/Logos/None
Generating bar plot...
No GroupBy Col
auto_title: False
testing
axes_data:{'y1': Index(['CRV comp', 'LINK comp', 'UNI comp', 'WBTC comp'], dtype='object'), 'y2': []}
sort_list: False
descending: True
columns to plot: ['WBTC comp', 'CRV comp', 'LINK comp', 'UNI comp']
tick0: 2024-12-16 07:00:00
reversed color map: {'WBTC comp': '#2E91E5', 'CRV comp': '#E15F99', 'LINK comp': '#1CA71C', 'UNI comp': '#FB0D0D'}
idx: 0, y: WBTC comp
auto_title: False
y1_lineto_show: None
index: DatetimeIndex(['2024-12-16 07:00:00', '2024-12-16 08:00:00',
               '2024-12-16 09:00:00', '2024-12-16 10:00:00',
               '2024-12-16 11:00:00', '2024-12-16 12:00:00',
               '2024-12-16 13:00:00', '2024-12-16 14:00:00',
               '2024-12-16 15:00:00', '2024-12-16 16:00:00',
               '2024-12-16 17:00:00', '2024-12-16 18:00:00',
               '2024-12-

aum_df:                      Portfolio Value
date                                
2024-12-16 07:00:00        51.037683
2024-12-16 22:00:00        50.263354
2024-12-17 03:00:00        50.263354
Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: 2024-12-16 07:00:00
turn_to_time: True
save:False
logo path: ../img/Logos/None
Generating line plot...
axes titles at viz pipeline: {'y1': None, 'y2': None}
No GroupBy Col
cols to plot: ['Portfolio Value']
axes data to plot: {'y1': Index(['Portfolio Value'], dtype='object'), 'y2': []}
tick0 in func: 2024-12-16 07:00:00
sort_list: False
axes_titles at beginning: {'y1': None, 'y2': None}
cumulative_sort: False
descending: True
columns to plot: ['Portfolio Value']
axes_font_colors: {'y1': 'black', 'y2': 'black'}
idx: 0 y1_col: Portfolio Value
axes_titles: {'y1': None, 'y2': None}
y1_lineto_show: None
axes_font_colors: {'y1': 'black', 'y2': 'black'}
latest val: 50.26335433654252
y1_col values:date
2024-12-16 07:00:00    51.037683
2024-12-16 22:00:

In [164]:
latest_comp_data = plot_historical_data.iloc[-1].to_frame('Composition').reset_index()

In [165]:
latest_comp_data = pd.DataFrame([model_balances_usd]).iloc[0].to_frame('Balance USD').reset_index()

In [ ]:
model_fig4 = visualization_pipeline(
    df=latest_comp_data,
    title='viz_port_values',
    chart_type='pie',
    groupby='index',
    num_col='Balance USD',
    show_legend=False,
    sort_list = False,
    legend_placement=dict(x=0.1,y=1.3),
    # annotation_prefix=dict(y1='$',y2=None),
    margin=dict(t=150,b=0,l=0,r=0),
    annotation_prefix='$',
    font_family=font_family
)

chartBuilder(
    fig=model_fig4,
    title='Latest Comp',
    dt_index=False,
    add_the_date=False,
    show=True,
    save=False
)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: min
turn_to_time: False
df sort order at init: ['CRV' 'LINK' 'UNI' 'WBTC']
save:False
logo path: ../img/Logos/None
Generating pie chart...
self.textinfo: percent+label
original_labels: ['CRV' 'LINK' 'UNI' 'WBTC']
df_copy:         Balance USD
index              
CRV    4.595670e-09
LINK   4.733659e+01
UNI    2.926769e+00
WBTC   0.000000e+00
percent+label, ['LINK    ', 'UNI    ', 'CRV    ', 'WBTC    ']
textinfo: percent+label
x:50.26335433654252
Columns: Index(['index', 'Balance USD'], dtype='object') 
Index: RangeIndex(start=0, stop=4, step=1)


In [ ]:
os.chdir('notebooks')